In [1]:
import os
import sys

import warnings

sys.path.append("..")

import critical_loads as cl

import matplotlib.pyplot as plt
import nivapy3 as nivapy
import numpy as np
import pandas as pd

warnings.simplefilter("ignore")
plt.style.use("ggplot")

# Updating critical loads for water

## Part E: Calculating critical loads

The original water chemistry data used to calculate critical loads for water is stored in the Critical Loads database. For periods before 1992 we will use the original values, but for periods from 1992 onwards we will use the updated values estimated in previous notebooks.

In [2]:
eng = nivapy.da.connect_postgis(database="critical_loads")

Connection successful.


## 1. Read updated BLR summaries

In [3]:
blr_df = pd.read_csv("updated_blr_summary.csv")
blr_df.rename({"cell_id": "Region_id"}, axis="columns", inplace=True)
blr_df.head()

,Region_id,NO3_NO2_ugpl_1990,NO3_NO2_ugpl_1991,NO3_NO2_ugpl_1992,NO3_NO2_ugpl_1993,NO3_NO2_ugpl_1994,NO3_NO2_ugpl_1995,NO3_NO2_ugpl_1996,NO3_NO2_ugpl_1997,NO3_NO2_ugpl_1998,...,NO3_NO2_ugpl_2018,TOC_mgpl_2019,NO3_NO2_ugpl_2019,runoff_mmpyr,Bare_area_km2,Forest_area_km2,Lake_area_km2,Other_area_km2,Peat_area_km2,Total_area_km2
0,58005004,210.943115,208.756673,205.685710,202.316915,199.028992,196.141479,193.593547,190.317057,186.298889,...,105.399445,5.223486,105.289591,1538.956424,0.0,7.948615,3.531350,41.628183,0.629654,53.737802
1,58006001,359.684215,355.956050,350.719671,344.975481,339.369191,334.445593,330.101022,324.514243,317.662760,...,179.719151,4.055069,179.531831,1815.066026,0.0,39.423553,31.021262,120.018516,2.302335,192.765666
2,58006002,196.554992,194.517683,191.656174,188.517168,185.453506,182.762957,180.388796,177.335823,173.591730,...,98.210299,3.599972,98.107946,2048.172561,0.0,76.189291,24.367177,97.210623,5.545539,203.312629
3,58006003,183.991995,182.084898,179.406288,176.467903,173.600081,171.081492,168.859086,166.001232,162.496440,...,91.933093,5.126954,91.837284,2013.350677,0.0,124.789407,36.497710,38.501417,3.524085,203.312618
4,58006004,146.026674,144.513088,142.387178,140.055128,137.779048,135.780159,134.016314,131.748162,128.966548,...,72.963417,6.559189,72.887360,1821.082721,0.0,161.622746,12.593858,16.768307,12.327718,203.312629


## 2. Estimate critical loads

In [4]:
st_yr = 1982
end_yr = 2017

# Loop over 5-year periods
for year in range(st_yr, end_yr + 1, 5):
    # Get start and end years for period
    st_per = year
    end_per = st_per + 4
    print(f"Processing {st_per} to {end_per}")

    # Get original data from database
    req_df, opt_df, mag_df = cl.read_orig_blr_data(eng, dropna=False)

    # Use the original data for the earliest periods, otherwise update
    if st_per >= 1992:
        # Update values
        req_df = cl.update_required_pars(req_df, blr_df, st_per, end_per)
        opt_df = cl.estimate_optional_pars(req_df, blr_df)

    # Calculate CLs for water
    cl_csv_path = f"../../water_cls/updated_cls_by_blr/updated_cls_by_blr_{st_per}-{str(end_per)[2:]}.csv"
    cl_df = cl.calculate_critical_loads_for_water(
        req_df=req_df, opt_df=opt_df, mag_df=mag_df, out_csv=cl_csv_path
    )

Processing 1982 to 1986
Processing 1987 to 1991
Processing 1992 to 1996
Processing 1997 to 2001
Processing 2002 to 2006
Processing 2007 to 2011
Processing 2012 to 2016
Processing 2017 to 2021


In [5]:
cl_df.describe()

,Region_id,Nimm_meq/m2/yr,Nupt_meq/m2/yr,rhoN_,Ffor_,CLrat_,BC0_ueq/l,BC0_Ffac_ueq/l,ANClimit_ueq/l,ANClimitOAA_ueq/l,...,Runoff_m/yr,ENO3_flux_meq/m2/yr,ECax_ueq/l,ECl_ueq/l,EMgx_ueq/l,ENax_ueq/l,ESO4x_ueq/l,ENO3_ueq/l,EKx_ueq/l,TOC_mg/l
count,2.469000e+03,2469.000000,2306.000000,2443.000000,2469.000000,2469.000000,2304.000000,2304.000000,2304.000000,2304.000000,...,2443.000000,2443.000000,2304.000000,2304.000000,2304.000000,2304.000000,2304.000000,2443.000000,2304.000000,2469.000000
mean,6.462676e+07,3.385949,2.559584,0.187436,0.359513,0.051555,142.794720,144.775581,21.865688,16.184931,...,1.224853,1.642180,101.559940,91.530643,30.739000,13.739078,46.127235,1.186829,5.080973,4.244834
std,4.097930e+06,0.187885,6.734782,0.162737,0.267451,0.052629,163.367689,174.622017,14.127143,12.054498,...,0.722210,2.692236,124.016449,103.586984,44.773670,22.518408,33.785190,1.743785,22.961239,4.540786
min,5.800500e+07,1.773874,0.000000,0.000000,0.000000,0.000000,6.284151,4.225902,1.458879,-1.993691,...,0.255051,0.012128,4.445298,2.820874,0.000000,0.000000,0.000000,0.034967,0.000000,0.143439
25%,6.100601e+07,3.325241,0.000000,0.066118,0.113151,0.017429,49.312679,44.111296,10.463220,6.654333,...,0.630095,0.193852,30.393402,22.566996,8.212791,0.000000,23.141070,0.223168,0.000000,1.234696
50%,6.401201e+07,3.440956,0.000000,0.138853,0.339414,0.036147,94.589368,91.905983,17.769678,12.697195,...,1.107969,0.655767,62.953807,53.596615,17.803211,9.296256,36.215092,0.562424,2.918602,2.374525
75%,6.901801e+07,3.507779,0.000000,0.259152,0.569311,0.068560,173.717645,174.329772,29.888412,23.117780,...,1.623612,1.684516,119.633247,126.939351,35.040269,19.575707,60.136561,1.368142,6.013011,5.504078
max,7.102801e+07,3.570000,50.000000,0.868409,0.969598,0.503117,2631.989107,2747.784604,50.000000,40.000000,...,4.486254,23.305900,1465.271685,792.665726,737.021799,662.070510,360.819368,15.822757,1013.370512,25.595052


In [6]:
req_df.describe()

,Region_id,Ca,Cl,K,Mg,Na,Nupt,SO4,Runoff,TOC,NO3N
count,2.469000e+03,2304.000000,2304.000000,1705.000000,2304.000000,2304.000000,2306.000000,2304.000000,2443.000000,2443.000000,2443.000000
mean,6.462676e+07,2.103129,3.244761,0.334845,0.591275,1.704553,2.559584,2.667132,1224.852786,4.279368,16.627470
std,4.097930e+06,2.492469,3.672159,1.045959,0.614808,2.064944,6.734782,1.706935,722.209712,4.552469,24.430435
min,5.800500e+07,0.110000,0.100000,0.020000,-0.097260,0.000000,0.000000,0.300000,255.051248,0.143439,0.489883
25%,6.100601e+07,0.670000,0.800000,0.150000,0.240000,0.330000,0.000000,1.500000,630.095066,1.254308,3.126582
50%,6.401201e+07,1.335000,1.900000,0.250000,0.440000,1.035000,0.000000,2.300000,1107.968750,2.419449,7.879553
75%,6.901801e+07,2.500000,4.500000,0.370000,0.750000,2.330000,0.000000,3.300000,1623.612428,5.551518,19.167668
max,7.102801e+07,29.500000,28.100000,40.000000,9.980000,23.800000,50.000000,18.000000,4486.253777,25.595052,221.676832
